# Using the subclassing API to build Dynamic Models

both the sequential api andd the functional API are declarative. you start by declaring which layers you want to use and how theyta should be connected. 

An advantae of this is that the model can easily be saved, cloned and shared: it's structure can be displayed and analyzed; the the framework can infer shapes ans check types so errors can be caught earlier. 

it is also faily easy to debug given the model is a static graph of layers. 

the disadvantage is just that it is static. A model may involve loops, varying shapes, conditional branching and other dynamic behaviors, for such cases,the subclassing API is needed

**How the subclass api works**

with this approach, you subclass the `Model` class, create layers needed in the constructor and use them to perform computations in the `call()` method. 

for example creating an instance of the following `wideAndDeepModel` class gives an equivalent to the functionapi model:

In [2]:

from sklearn.datasets import fetch_california_housing
import tensorflow as tf

class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) #needed to support naming the model

        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1)
        self.aux_output = tf.keras.layers.Dense(1)

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.concatenate([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output

model = WideAndDeepModel(30, activation="relu", name="my_cool_model")      
        